In [31]:
import os
import nncf
import zipfile
import requests
import urllib.parse
from pathlib import Path
from tqdm.notebook import tqdm_notebook

INFO:nncf:NNCF initialized successfully. Supported frameworks detected: torch, tensorflow, onnx, openvino


In [30]:
# download tiny imagenet 200 dataset
dataset_url = "http://cs231n.stanford.edu/tiny-imagenet-200.zip"

data_dir = Path("dataset")

if not data_dir.exists():
    data_dir.mkdir(parents = True, exist_ok = True)

tarname = Path(urllib.parse.urlparse(dataset_url).path).name
archive_path = data_dir / tarname

chunk_size = 16384

filename = data_dir / Path(tarname)

try:
    response = requests.get(url = dataset_url, headers = {"User-agent": "Mozilla/5.0"}, stream = True)
    response.raise_for_status()
except requests.exceptions.HTTPError as error:
    raise Exception(error) from None
except requests.exceptions.Timeout:
    raise Exception(
        "connection timed out error."
    ) from None
except requests.exceptions.RequestException as error:
    raise Exception(f"file downloading failed with error: {error}") from None

filesize = int(response.headers.get("Content-length", 0))

if not filename.exists() or (os.stat(filename).st_size != filesize):
    with tqdm_notebook(
        total = filesize,
        unit = "B",
        unit_scale = True,
        unit_divisor = 1024,
        desc = str(filename),
        disable = False,
    ) as progress_bar:
        with open(filename, "wb") as file_object:
            for chunk in response.iter_content(chunk_size):
                file_object.write(chunk)
                progress_bar.update(len(chunk))
                progress_bar.refresh()
else:
    print(f"'{filename}' already exists.")

response.close()

absolute_path = filename.resolve()
print(absolute_path)

if not archive_path.exists():
    zip_ref = zipfile.ZipFile(archive_path, "r")
    zip_ref.extractall(path = data_dir)
    zip_ref.close()
else:
    print(f"'{archive_path}' already extracted.")

'dataset\tiny-imagenet-200.zip' already exists.
C:\Users\huchri1x\Desktop\git\computer-vision-deep-learning-project\project\openvino\dataset\tiny-imagenet-200.zip
'dataset\tiny-imagenet-200.zip' already extracted.
